In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
keras = tf.keras
from scipy import stats
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
mpl.rcParams['font.size'] = 14
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['figure.titlesize'] = 'large'

In [20]:
X_train = pd.read_csv('../data/train_clean.csv', index_col='Date').drop(columns='Unnamed: 0')
X_test = pd.read_csv('../data/test_clean.csv', index_col='Date').drop(columns='Unnamed: 0')

In [21]:
X_train.head()

,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_x0_A,Type_x0_B,Type_x0_C,Month_x0_1,Month_x0_2,Month_x0_3,Month_x0_4,Month_x0_5,Month_x0_6,Month_x0_7,Month_x0_8,Month_x0_9,Month_x0_10,Month_x0_11,Month_x0_12,Store,IsHoliday,Weekly_Sales
Date,,,,,,,,,,,,,,,,,,,,,,,
2010-02-05 00:00:00,-0.956241,-1.694130,1.003435,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1643690.90
2010-02-12 00:00:00,-1.160856,-1.746757,1.007199,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,True,1641957.44
2010-02-19 00:00:00,-1.084394,-1.821313,1.008412,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1611968.17
2010-02-26 00:00:00,-0.723626,-1.718251,1.009200,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1409727.59
2010-03-05 00:00:00,-0.730626,-1.577911,1.009987,0.167116,0.347697,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1554806.68


In [22]:
X_train.columns

Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Type_x0_A',
       'Type_x0_B', 'Type_x0_C', 'Month_x0_1', 'Month_x0_2', 'Month_x0_3',
       'Month_x0_4', 'Month_x0_5', 'Month_x0_6', 'Month_x0_7', 'Month_x0_8',
       'Month_x0_9', 'Month_x0_10', 'Month_x0_11', 'Month_x0_12', 'Store',
       'IsHoliday', 'Weekly_Sales'],
      dtype='object')

In [6]:
X_train.index = pd.to_datetime(X_train.index)
X_test.index = pd.to_datetime(X_test.index)

In [9]:
piv = X_train[['Store', 'Weekly_Sales']].pivot_table('Weekly_Sales', X_train.index,'Store')
# piv.fillna(0, inplace=True)
piv

Store,1,2,4,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,29,30,32,33,34,35,36,37,38,39,40,41,42,43,45
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1643690.90,2136989.46,2135143.87,1652635.10,496725.44,1004137.09,549505.55,2193048.75,1528008.64,1967220.53,2623469.95,652122.44,477409.30,789036.02,1205307.50,1507637.17,2401395.47,798593.88,1033017.37,1364721.58,1034119.21,538634.46,465108.52,1087616.19,274593.43,956228.96,1230613.50,467546.74,536006.73,358496.14,1230596.80,1001943.80,1086533.18,543384.01,647029.28,890689.51
2010-02-12,1641957.44,2137809.50,2188307.39,1606283.86,524104.92,994801.40,552677.48,2176028.52,1574684.08,2030933.46,1704218.84,682447.10,472044.28,841951.91,1187880.70,1536549.95,2109107.90,809321.44,1022571.25,1380892.08,1015684.09,529672.95,497374.57,1123566.12,294882.83,994610.99,1168815.31,469563.70,529852.70,342214.90,1266229.07,955338.29,1075656.34,575709.96,682918.99,656988.64
2010-02-19,1611968.17,2124451.54,2049860.26,1567138.07,506760.54,963960.37,511327.90,2113432.58,1503298.70,1970274.64,2204556.70,660838.75,469868.70,800714.00,1150663.42,1515976.11,2161549.76,867283.25,988467.61,1319588.04,999348.55,542399.07,463513.26,1082559.06,296850.83,983963.07,1270658.64,470281.03,510382.50,327237.92,1230591.97,916289.20,1052034.74,508794.87,658997.55,841264.04
2010-02-26,1409727.59,1865097.27,1925728.84,1432953.21,496083.24,847592.11,473773.27,2006774.96,1336404.65,1817850.32,2095591.63,564883.20,443242.17,749549.55,1068157.45,1373270.06,1898193.95,749597.24,899761.48,1198709.65,855385.01,488417.61,472330.71,1053247.10,284052.77,905756.13,1020651.74,447519.44,513615.82,334222.73,1168582.02,863917.41,991941.73,491510.58,618702.79,741891.65
2010-03-05,1554806.68,1991013.13,1971057.44,1601348.82,491419.55,881503.95,507297.88,1987090.09,1426622.65,1939980.43,2237544.75,605325.43,444181.85,783300.05,1179738.50,1495844.57,2119213.72,747444.32,1009201.24,1311175.93,1005669.58,535087.91,472591.07,1066566.74,291484.89,918295.79,1162610.27,480203.43,519255.68,372239.89,1266254.21,990152.28,1063557.49,554972.42,658600.05,777951.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-09-28,1437059.26,1746470.56,2027620.23,1369131.46,525545.76,884724.41,516361.06,1694862.41,1227430.73,1927664.11,1522512.20,553901.97,469607.73,972716.24,1002856.20,1338299.02,2008350.58,647097.65,976479.51,1319035.06,964726.37,489674.23,425410.04,1157111.15,242813.51,928629.31,814099.86,270677.98,527953.14,398468.08,1495607.07,919595.44,1307928.01,505978.46,577792.32,713173.95
2012-10-05,1670785.97,1998321.04,2209835.43,1518177.71,505830.56,976436.02,606755.30,1758971.38,1422794.26,2041918.74,1687592.16,573498.64,471281.68,952609.17,1092204.79,1408016.10,2246411.89,651768.91,1009887.36,1464616.59,1095504.26,520632.80,446751.45,1202775.24,265444.90,968896.68,866064.40,277137.86,546122.37,458479.01,1574408.67,1069112.00,1400160.95,593162.53,642614.89,733455.07
2012-10-12,1573072.81,1900745.13,2133026.07,1459396.84,503463.93,927511.99,558464.80,1713889.11,1311965.09,1999079.44,1639585.61,551799.63,491817.19,919878.34,1074079.00,1352809.50,2162951.36,653043.44,1004039.84,1412925.25,1044639.69,513737.00,434593.26,1176681.31,291781.15,948613.39,873643.14,300236.85,521810.75,437320.66,1494417.07,982523.26,1409544.97,612379.90,619369.72,734464.36


In [12]:
def windowize_data(data, n_prev, n_future, create_y=False):
    '''
    returns:
        x: np array with shape (n_predictions, n_prev, 1)
        y: np array with shape (n_predictions, n_future, 1)
    '''
    n_predictions = len(data) - n_prev - n_future +1
    
    x_indices = np.arange(n_prev) + np.arange(n_predictions)[:, None]
    x = data[x_indices, None]

    if create_y:
        y = data[n_prev:]
        dates = data.index[n_prev:]
        y_indices = np.arange(n_future) + np.arange(n_predictions)[:, None]
        y = y[y_indices, None]
        y_dates = dates[y_indices, None]

        return x, y, y_dates
    else:
        return x

def window_multi_groups(data, n_prev, n_future, create_y=False):
    '''
    returns:
        x: np array with shape (n_predictions * number of stores, n_prev, 1)
        y: np array with shape (n_predictions * number of stores, n_future, 1)
    '''
    ncols = data.shape[1]
    n_pred = len(data) - n_prev - n_future +1
    n_total_pred = n_pred * ncols
    x = np.zeros((n_total_pred, n_prev, 1))
    if create_y:
        y = np.zeros((n_total_pred, n_future, 1))
        y_dates = np.empty((n_total_pred, n_future, 1),dtype='datetime64[ns]')
        for i in range(ncols):
            start = i * n_pred
            stop = start + n_pred
            x[start:stop, :, :], y[start:stop, :, :], y_dates[start:stop, :, :] = windowize_data(data.iloc[:,i], n_prev, n_future, create_y=create_y)
        return x, y, y_dates
    else:
        for i in range(ncols):
            start = i * n_pred
            stop = start + n_pred
            x[start:stop, :, :]= windowize_data(data.iloc[:,i], n_prev, n_future, create_y=create_y)
        return x

def window_multivar(data, n_prev, n_future, feats, target='Weekly_Sales'):
    '''
    returns:
        x: np array with shape (n_predictions * number of stores, n_prev, n_feats)
        y: np array with shape (n_predictions * number of stores, n_future, 1)
        y_dates: np array with shape (n_predictions * number of stores, n_future, 1)
    '''
    piv = data[['Store', target]].pivot_table(target, data.index, 'Store')
    X, y, y_dates = window_multi_groups(piv, n_prev, n_future, create_y=True)
    if feats != None:
        for feat in feats:
            piv = data[['Store', feat]].pivot_table(feat, data.index, 'Store')
            X_feat = window_multi_groups(piv, n_prev, n_future)
            X = np.dstack((X, X_feat))
    return X, y, y_dates

In [14]:
feats = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Month_x0_11', 'Month_x0_12', 'IsHoliday']

In [17]:
window_multivar(X_train, 3, 2, feats)[0][0]

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:16: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:17: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Support for multi-d

array([[ 1.64369090e+06, -9.56240592e-01, -1.69412982e+00,
         1.00343503e+00,  1.67116416e-01,  3.47696689e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.64195744e+06, -1.16085570e+00, -1.74675725e+00,
         1.00719943e+00,  1.67116416e-01,  3.47696689e-01,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.61196817e+06, -1.08439426e+00, -1.82131278e+00,
         1.00841213e+00,  1.67116416e-01,  3.47696689e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [18]:
X_train.head(10)

,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_x0_A,Type_x0_B,Type_x0_C,Month_x0_1,Month_x0_2,Month_x0_3,Month_x0_4,Month_x0_5,Month_x0_6,Month_x0_7,Month_x0_8,Month_x0_9,Month_x0_10,Month_x0_11,Month_x0_12,Store,IsHoliday,Weekly_Sales
Date,,,,,,,,,,,,,,,,,,,,,,,
2010-02-05,-0.956241,-1.694130,1.003435,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1643690.90
2010-02-12,-1.160856,-1.746757,1.007199,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,True,1641957.44
2010-02-19,-1.084394,-1.821313,1.008412,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1611968.17
2010-02-26,-0.723626,-1.718251,1.009200,0.167116,0.347697,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1409727.59
2010-03-05,-0.730626,-1.577911,1.009987,0.167116,0.347697,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1554806.68
2010-03-12,-0.122703,-1.485813,1.010774,0.167116,0.347697,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1439541.59
2010-03-19,-0.295549,-1.369594,1.006514,0.167116,0.347697,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1472515.79
2010-03-26,-0.464087,-1.343280,1.001413,0.167116,0.347697,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1404429.92
2010-04-02,0.118527,-1.371787,0.996312,-0.016967,0.347697,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,1594968.28


In [24]:
X_train[['Store', 'Temperature']].pivot_table('Temperature', X_train.index,'Store')

Store,1,2,4,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,29,30,32,33,34,35,36,37,38,39,40,41,42,43,45
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-02-05 00:00:00,-0.956241,-1.070394,-0.878164,-1.057471,-2.667469,-1.396163,-1.187779,-0.308472,-0.755395,-1.536701,-1.763932,-2.166700,-2.168854,-1.990085,-2.085931,-2.105854,-1.838778,-1.131779,-1.922778,-2.413316,-2.720238,-1.922778,-1.131779,-1.380548,-0.089857,-1.326163,-1.770393,-0.759164,-0.759164,-0.570703,-0.849087,-2.454777,-1.604547,-0.308472,-0.687010,-1.763932
2010-02-12 00:00:00,-1.160856,-1.161933,-1.681547,-1.049933,-1.839855,-1.439240,-1.237856,-0.544318,-0.649318,-1.448932,-1.741316,-2.049854,-2.110701,-2.245854,-1.812393,-1.984162,-2.043393,-1.200702,-1.719240,-2.224854,-2.257700,-1.719240,-1.200702,-1.721932,-0.247626,-1.289009,-1.629317,-0.751625,-0.751625,-0.656856,-0.834010,-2.112316,-1.993855,-0.544318,-0.650395,-1.741316
2010-02-19 00:00:00,-1.084394,-1.097317,-1.271779,-0.887856,-1.765547,-1.129087,-0.915856,-0.099549,-0.633703,-1.312163,-1.550701,-1.751547,-2.096701,-1.885085,-1.570086,-1.696086,-1.865162,-1.094087,-1.513548,-1.796778,-2.016470,-1.513548,-1.094087,-1.664317,0.112604,-1.168933,-1.487701,-0.775856,-0.775856,-0.282088,-0.867395,-1.735393,-1.935162,-0.099549,-0.608395,-1.550701
2010-02-26 00:00:00,-0.723626,-0.752164,-1.007394,-0.698318,-1.893162,-1.193163,-0.874395,-0.393011,-0.391934,-1.620163,-1.355779,-1.626086,-2.258777,-2.392316,-1.327240,-1.601317,-1.494163,-0.794702,-1.297086,-1.474778,-1.763393,-1.297086,-0.794702,-1.800009,-0.192165,-1.227086,-1.296009,-0.495318,-0.495318,-0.529780,-0.553472,-1.440317,-2.058470,-0.393011,-0.608395,-1.355779
2010-03-05 00:00:00,-0.730626,-0.694549,-0.892702,-0.562087,-1.303548,-0.776933,-0.626703,-0.223395,-0.328934,-1.045625,-1.235163,-1.522701,-1.731086,-1.534009,-1.233009,-1.461855,-1.524855,-0.617010,-1.251856,-1.449471,-1.694470,-1.251856,-0.617010,-1.203394,-0.048396,-0.937933,-1.184548,-0.470549,-0.470549,-0.339165,-0.492087,-1.361702,-1.485009,-0.223395,-0.386549,-1.235163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-09-28 00:00:00,0.862142,1.043603,0.628450,0.955834,-0.507703,0.593988,0.900911,1.208910,0.947757,0.254758,0.259065,-0.158242,-0.466780,-0.123780,-0.098473,-0.175472,-0.076396,1.093680,0.113143,-0.342395,-0.506626,0.113143,1.093680,0.029681,1.418910,0.376450,0.261219,1.045757,1.045757,1.138910,0.967680,-0.489395,-0.214780,1.208910,0.941295,0.259065
2012-10-05 00:00:00,0.456681,0.549296,0.161604,0.579988,-0.626703,0.072220,0.352219,1.120603,0.716219,0.092681,0.259604,-0.026857,-0.517395,-0.390857,-0.020934,-0.011242,0.037758,0.549834,0.108835,-0.184088,-0.323011,0.108835,0.549834,-0.254626,1.352141,0.287604,0.238604,0.726988,0.726988,1.159910,0.645142,-0.153934,-0.534626,1.120603,0.754988,0.259604
2012-10-12 00:00:00,0.157296,0.048527,-0.159319,0.288681,-1.003625,-0.271318,0.001143,0.859449,0.531527,-0.267549,-0.301472,-0.589549,-0.905087,-0.821087,-0.489934,-0.634241,-0.692933,0.078681,-0.317088,-0.644472,-0.801164,-0.317088,0.078681,-0.892702,1.053834,-0.006934,-0.251395,0.603680,0.603680,0.628450,0.528296,-0.684856,-1.114010,0.859449,0.596142,-0.301472


In [ ]:
windowize_data(piv.iloc[:,1], 3, 2)[0].shape

In [18]:
windowize_data(piv.iloc[:7,1], 3, 2)


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:16: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


(array([[[24213.18],
         [21760.75],
         [18706.21]],
 
        [[21760.75],
         [18706.21],
         [17306.61]],
 
        [[18706.21],
         [17306.61],
         [19082.9 ]]]),
 array([[[17306.61],
         [19082.9 ]],
 
        [[19082.9 ],
         [17864.32]],
 
        [[17864.32],
         [19738.42]]]),
 array([[['2010-02-26T00:00:00.000000000'],
         ['2010-03-05T00:00:00.000000000']],
 
        [['2010-03-05T00:00:00.000000000'],
         ['2010-03-12T00:00:00.000000000']],
 
        [['2010-03-12T00:00:00.000000000'],
         ['2010-03-19T00:00:00.000000000']]], dtype='datetime64[ns]'))

In [32]:
piv.iloc[:7,1]

Date
2010-02-05    24213.18
2010-02-12    21760.75
2010-02-19    18706.21
2010-02-26    17306.61
2010-03-05    19082.90
2010-03-12    17864.32
2010-03-19    19738.42
Name: 1-11, dtype: float64